In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/MyDrive"

In [ ]:
cd "/content/drive/MyDrive/selection-of-criminals_with_syndata/CTGAN_syndata_generaton" 

In [ ]:
!pip install sdmetrics

# 3. Review generated data similarity(Coverage)

 ## Data load

In [1]:
import pandas as pd
import copy

In [2]:
df_syn_01 = pd.read_csv('./data_sample/df_syn_en_14.csv', encoding='utf-8-sig') #새로운 가상데이터

FileNotFoundError: [Errno 2] No such file or directory: './data_sample/df_syn_en_09.csv'

In [ ]:
df_syn_base = pd.read_csv('df_syn_en.csv', encoding='utf-8-sig') #기존 가상데이터

In [ ]:
copy_syn = copy.deepcopy(df_syn_01)

In [ ]:
copy_base = copy.deepcopy(df_syn_base)

## 생성된 데이터의 기존 데이터 포함 범위 확인

### 카테고리 타입 데이터 커버리지 확인

This metric measures whether a synthetic column covers all the possible categories that are present in a real column.

Data Compatibility
Categorical: This metric is meant for discrete, categorical data
Boolean: This metric is meant for boolean data

Score
- (best) 1.0: The synthetic column contains all the unique categories present in the real column
- (worst) 0.0: The synthetic column contains none of the categories present in the real column
* The plot below shows some fictitious real and synthetic data (black and green respectively) with CategoryCoverage=0.6.

![Category Coverage](https://1485348715-files.gitbook.io/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2FrNLha4DaPNwVJ930KhmB%2Fuploads%2F2yn1OQLODOesNLVjnpe2%2FCategory%20Coverage.png?alt=media&token=e12f6f08-d3b0-4e05-9a41-58b8a28d4a4e)

How does it work?
This metric first computes the number of unique categories, c, that are present in the real column r. Then it computes the number of those categories present in the synthetic column, s. It returns the proportion of real categories that are in the synthetic data.

In [ ]:
# 'dec_custom_code' 칼럼의의 커버리지 점수를 구합니다. 
from sdmetrics.single_column import CategoryCoverage

CategoryCoverage.compute(
    real_data=copy_base['dec_custom_code'],
    synthetic_data=copy_syn['dec_custom_code']
)

### 카테고리 타입의 항목별 커버리지 그래프 그리기

- 카테고리 타입의 항목별 커버리지 점수를 저장할 데이터 프레임을 생성합니다. 

In [ ]:
# 카테고리 타입의 항목별 커버리지 점수를 저장할 데이터 프레임을 생성합니다. 
CC_score = pd.DataFrame({'Category':[],'Score':[]})

#카테고리 타입의 칼럼을 변수에 입력합니다. 
Range_columns = ['dec_num','dec_custom_code','imp_dec_code','imp_trd_code','imp_typ_code',\
                 'collect_code','typ_transport_code','dec_mark','importer','ovs_cust_code',\
                 'exps_carr_code','HS10','country_ship_code','country_orig_code','trff_class_code',\
                 'country_orig_mark_code','crime_yn','key_exposure']

#원본 데이터의 인덱스와 값을 가지고 가상데이터와의 커버리지 점수를 구하여 데이터 프렝임에 저장합니다. 
for idx,val in enumerate(copy_base):
    score = 0 
    if val in Range_columns:
        score = CategoryCoverage.compute(
            real_data=copy_base[val],
            synthetic_data=copy_syn[val]
        )
    CC_score.loc[idx] = [val,score]

- 커버리지 점수가 담긴 데이터 프레임을 그래프화 하기 위한 라이브러리를 로드 합니다. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

- 계산된 커버리지 점수를 활용해 막대 그래프를 그립니다. 

In [ ]:
#카테고리 데이터의 수식상 음수는 나올 수 없기 때문에 data 변수에 음수를 제거하여 입력하는 조건을 붙였습니다. 
sns.barplot(x='Category',y ='Score', data = CC_score[CC_score['Score'] > 0])
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 10
plt.xticks(rotation =70)

### 수치형 타입  데이터 커버리지 확인

This metric measures whether a synthetic column covers the full range of values that are present in a real column.

Data Compatibility
- Numerical : This metric is meant for continuous, numerical data
- Datetime : This metric converts datetime values into numerical values

Score
- (best) 1.0: The synthetic column covers the range of values present in the real column
- (worst) 0.0: The synthetic column does not overlap at all with the range of values in the real column
* The plot below shows some fictitious real and synthetic data (black and green respectively) with RangeCoverage=0.82.

![Value](https://1485348715-files.gitbook.io/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2FrNLha4DaPNwVJ930KhmB%2Fuploads%2FUa21ikMRPxduAFIpovjM%2FRange%20Coverage.png?alt=media&token=bb7dca22-ad7d-4521-8523-e683588deb2b)

In [ ]:
# 'trff_rate' 칼럼의의 커버리지 점수를 구합니다. 
from sdmetrics.single_column import RangeCoverage

RangeCoverage.compute(
    real_data=copy_base['trff_rate'],
    synthetic_data=copy_syn['trff_rate']
)

### 수치형 타입의 항목별 커버리지 그래프 그리기

- 수치형 타입의 항목별 커버리지 점수를 저장할 데이터 프레임을 생성합니다. 

In [ ]:
# 수치형 타입의 항목별 커버리지 점수를 담을 데이터 프레임을 생성합니다. 
RC_score = pd.DataFrame({'Category':[],'Score':[]})

#수치형 타입의 칼럼을 변수에 입력합니다. 
Range_columns = ['trff_rate','dec_weight','taxabal_price_KRW']

#원본 데이터의 인덱스와 값을 가지고 가상데이터와의 커버리지 점수를 구하여 데이터 프렝임에 저장합니다. 
for idx,val in enumerate(copy_base):
    score = 0 
    if val in Range_columns:
        score = RangeCoverage.compute(
            real_data=copy_base[val],
            synthetic_data=copy_syn[val]
        )
    RC_score.loc[idx] = [val,score]

- 계산된 커버리지 점수를 활용해 막대 그래프를 그립니다. 

In [ ]:
sns.barplot(x='Category',y ='Score', data = RC_score[RC_score['Score'] > 0])
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 10
plt.xticks(rotation =70)

## 결과 통합하기

### 두개의 결과 데이터 프레임을 결합하기

- CC_score의 'trff_rate','dec_weight','taxabal_price_KRW' 칼럼에 RC_score가 가지고 있는 값을 대입한다. 

In [ ]:
CC_score.loc[[15,18,19],'Score'] = RC_score.loc[[15,18,19],'Score']

- 전체 score를 확인한다. 

In [ ]:
Total_score = CC_score

In [ ]:
plt.figure(figsize=(10,10))
colors = sns.color_palette('Greys', 1)
sns.barplot(x='Category',y ='Score', data = Total_score[Total_score['Score'] > 0])
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 10
plt.xticks(rotation =70)